<a href="https://colab.research.google.com/github/Vijaycriss/MNSIT_Tensorflow_DNN/blob/main/MNSIT_DNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds


##Loading the MNIST Dataset

In [2]:
mnist_data, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

Dl Completed...:   0%|          | 0/5 [00:00<?, ? file/s]

Dataset mnist downloaded and prepared to /root/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


##Spliting tht dataset and scaling it

In [3]:
mnist_train, mnist_test = mnist_data['train'], mnist_data['test']

num_val_samples = 0.1 * mnist_info.splits['train'].num_examples

num_val_samples = tf.cast(num_val_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples

num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
  image = tf.cast(image, tf.float32)
  image /= 255.
  return image, label

scaled_train_and_val_data = mnist_train.map(scale)

scaled_test_data = mnist_test.map(scale)

Shuffling and Batching

In [14]:
#Shuffle
BUFFER_SIZE = 10000

shuffle_train_and_val_data = scaled_train_and_val_data.shuffle(BUFFER_SIZE)

validation_data = scaled_train_and_val_data.take(num_val_samples)

train_data = shuffle_train_and_val_data.skip(num_val_samples)

#Batch
BATCH_SIZE = 1000

train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_val_samples)
test_data = scaled_test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

##Outline the model

In [15]:
input_size = 784
output_size = 10
hidden_layers = 2000

model = tf.keras.Sequential([
                          tf.keras.layers.Flatten(input_shape = (28,28,1)),
                          tf.keras.layers.Dense(hidden_layers, activation = 'relu'),
                          tf.keras.layers.Dense(hidden_layers, activation = 'relu'),
                          tf.keras.layers.Dense(hidden_layers, activation = 'relu'),
                          tf.keras.layers.Dense(hidden_layers, activation = 'relu'),

                          tf.keras.layers.Dense(output_size, activation='softmax')
])

#Optimizer and Loss Function

In [16]:
model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

##Training the model

In [17]:
NUMs_EPOCH = 5

model.fit(train_data, epochs = NUMs_EPOCH, validation_data = (validation_inputs, validation_targets), verbose= 2 )

Epoch 1/5
54/54 - 98s - loss: 0.3880 - accuracy: 0.8750 - val_loss: 0.1182 - val_accuracy: 0.9650 - 98s/epoch - 2s/step
Epoch 2/5
54/54 - 96s - loss: 0.0861 - accuracy: 0.9729 - val_loss: 0.0735 - val_accuracy: 0.9800 - 96s/epoch - 2s/step
Epoch 3/5
54/54 - 97s - loss: 0.0460 - accuracy: 0.9859 - val_loss: 0.0679 - val_accuracy: 0.9800 - 97s/epoch - 2s/step
Epoch 4/5
54/54 - 101s - loss: 0.0337 - accuracy: 0.9894 - val_loss: 0.0474 - val_accuracy: 0.9860 - 101s/epoch - 2s/step
Epoch 5/5
54/54 - 96s - loss: 0.0300 - accuracy: 0.9900 - val_loss: 0.0301 - val_accuracy: 0.9905 - 96s/epoch - 2s/step


##Test the model

In [18]:
test_loss, test_accuracy = model.evaluate(test_data)
print('Test loss : {0:.2f}, Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100))

1/1 [==============================] - 8s 8s/step - loss: 0.0709 - accuracy: 0.9797
Test loss : 0.07, Test accuracy: 97.97%
